In [1]:
from readers import read_timeml
from datasets import DatasetDict

d:\GeoTKG\GeoTKG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
directory_path = './rawdata/Tempeval3/Training/TE3-Silver-data-0'
intended_path = './cleandata/Tempeval3/silver-1.json'
train_dataset, label_list, label2id, id2label = read_timeml(directory_path, intended_path)

Processing file: AFP_ENG_19970401.0006.tml
Processing file: AFP_ENG_19970401.0092.tml
Processing file: AFP_ENG_19970401.0099.tml
Processing file: AFP_ENG_19970401.0129.tml
Processing file: AFP_ENG_19970401.0171.tml
Processing file: AFP_ENG_19970401.0222.tml
Processing file: AFP_ENG_19970401.0394.tml
Processing file: AFP_ENG_19970401.0421.tml
Processing file: AFP_ENG_19970401.0445.tml
Processing file: AFP_ENG_19970401.0534.tml
Processing file: AFP_ENG_19970401.0541.tml
Processing file: AFP_ENG_19970401.0562.tml
Processing file: AFP_ENG_19970402.0016.tml
Processing file: AFP_ENG_19970402.0094.tml
Processing file: AFP_ENG_19970402.0123.tml
Processing file: AFP_ENG_19970402.0189.tml
Processing file: AFP_ENG_19970402.0207.tml
Processing file: AFP_ENG_19970402.0208.tml
Processing file: AFP_ENG_19970402.0278.tml
Processing file: AFP_ENG_19970402.0349.tml
Processing file: AFP_ENG_19970402.0390.tml
Processing file: AFP_ENG_19970402.0422.tml
Processing file: AFP_ENG_19970402.0427.tml
Processing 

Creating json from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 164.79ba/s]


In [3]:
directory_path = './rawdata/Tempeval3/Evaluation/te3-platinum'
intended_path = './cleandata/Tempeval3/platinum.json'
eval_dataset, label_list, label2id, id2label = read_timeml(directory_path, intended_path)

Processing file: AP_20130322.tml
Processing file: bbc_20130322_1150.tml
Processing file: bbc_20130322_1353.tml
Processing file: bbc_20130322_1600.tml
Processing file: bbc_20130322_332.tml
Processing file: bbc_20130322_721.tml
Processing file: CNN_20130321_821.tml
Processing file: CNN_20130322_1003.tml
Processing file: CNN_20130322_1243.tml
Processing file: CNN_20130322_248.tml
Processing file: CNN_20130322_314.tml
Processing file: nyt_20130321_china_pollution.tml
Processing file: nyt_20130321_cyprus.tml
Processing file: nyt_20130321_sarcozy.tml
Processing file: nyt_20130321_women_senate.tml
Processing file: nyt_20130322_strange_computer.tml
Processing file: WSJ_20130318_731.tml
Processing file: WSJ_20130321_1145.tml
Processing file: WSJ_20130322_159.tml
Processing file: WSJ_20130322_804.tml


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 444.50ba/s]


In [4]:
datasets = DatasetDict({
        "train": train_dataset,
        "eval": eval_dataset
    })

In [20]:
from model import Model
import torch
from transformers import RobertaForTokenClassification, RobertaTokenizerFast, TrainingArguments

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Switching to GPU...")
    device = torch.device('cuda')
print("Current device index:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Switching to GPU...
Current device index: 0
Device name: NVIDIA GeForce RTX 4070 Ti


In [17]:
# Load tokenizer and model
model_name = 'roberta-base'
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=label2id, id2label=id2label)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_instance = Model(
    device=device,
    model_name=model_name,
    tokenizer=tokenizer,
    model=model,
    label_list=label_list
)

model_instance.set_training_args(training_args)
tokenized_datasets = model_instance.tokenize_datasets(datasets)

Tokenizing datasets...


Map: 100%|██████████| 258/258 [00:00<00:00, 4555.80 examples/s]


In [24]:
model_instance.train(tokenized_datasets)


Starting training...


Step,Training Loss,Validation Loss,Precision,Recall,F1
100,0.026400,0.050306,0.776860,0.681159,0.725869
200,0.021200,0.043933,0.750000,0.717391,0.733333
